In [ ]:
# Import python packages
import streamlit as st

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Define image in a stage and read the file
image=session.file.get_stream('@aicollege.public.setup/SnowflakeML.jpg' , decompress=False).read() 

# Display the image
st.image(image, width=800)

### ❄️ End-to-End ML Retraining with Snowflake and Open-Source Models ❄️

This section walks through the full retraining pipeline using **Snowflake-native data**, **open-source ML libraries**, and the **Snowflake Model Registry** to version, monitor, and explain updated model behavior.

Before running this notebook, make sure the enriched dataset has been loaded into the `NEWTRAININGDATA` table. This table combines historical and recent mortgage data to improve generalization and reduce drift.

In this retraining pipeline, we will:

- Load and explore the enriched data from the `NEWTRAININGDATA` table  
- Preprocess inputs and engineer features  
- Register features into the **Snowflake Feature Store**
- Retrain a model using **open-source ML frameworks** like `xgboost` or `scikit-learn`
- Register the updated model as **Version 2** in the **Snowflake Model Registry**
- Set up **ML Monitors** to compare V1 vs. V2 on performance drift
- Use **Snowflake ML Explainability** (`EXPLAIN`) to generate SHAP-style insights
- Promote the best-performing model version to production using **aliases or version control**
- Run **dynamic batch scoring** in Python **using the production alias**

> ✅ This notebook assumes your original model (Version 1) is already registered as `COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL` in the Snowflake Model Registry.

In [ ]:
-- Validate query COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL version V1 is registered in Snowflake Model Registry
SHOW VERSIONS IN MODEL COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL;

### 📥 Load and Inspect the Enriched Training Dataset

In this step, we load the `NEWTRAININGDATA` table, which contains a curated dataset designed to support retraining on more current and representative patterns. It includes:

- Original training data capturing historical trends
- Recent labeled examples from production (e.g., batch inference results with ground truth)
- Optionally, synthetic records to match the distribution of current input patterns

Rather than using a simple `SELECT * ... LIMIT`, we’ll preview the data using a query that returns a **sample of rows per `LOAN_PURPOSE_NAME`**. This approach gives a more balanced snapshot of the dataset across different loan purposes, which helps validate readiness for downstream preprocessing and feature engineering.

In [ ]:
SELECT * EXCLUDE (rn)
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY LOAN_TYPE_NAME ORDER BY LOAN_ID) AS rn
    FROM AICOLLEGE.PUBLIC.NEWTRAININGDATA
)
WHERE rn <= 2;

### 🧹 Data Cleaning & Sanitization

Before we apply One-Hot Encoding or register features to the Snowflake Feature Store, we create a cleaned and standardized version of the training data called `NEWTRAININGDATA_CLEANED`. This ensures downstream steps operate on consistent, ML-friendly inputs.

In this step, we:

- **Map known loan-type values**  
  Standardize strings for model compatibility:  
  - `FSA/RHS-guaranteed` → `FSA_RHS`  
  - `FHA-insured` → `FHA`  
  - `VA-guaranteed` → `VA`  

- **Normalize key categorical fields**  
  - `LOAN_PURPOSE_NAME`: Replace hyphens, spaces, and periods with underscores (e.g., `home-improvement` → `home_improvement`)  
  - `COUNTY_NAME`: Remove hyphens, underscores, and periods entirely (e.g., `St. Lawrence County` → `St Lawrence County`)  

- **Preserve temporal and ID metadata**  
  Retain `WEEK_START_DATE`, `WEEK`, `LOAN_ID`, and `TS` fields for observability and feature tracking.

- **Ensure correct numeric types**  
  Explicitly cast `APPLICANT_INCOME_000S` and `LOAN_AMOUNT_000S` as `FLOAT` to prevent type issues during feature engineering or training.

The resulting `NEWTRAININGDATA_CLEANED` table provides a consistent and sanitized foundation for one-hot encoding, feature store registration, and model retraining.

In [ ]:
CREATE OR REPLACE TABLE AICOLLEGE.PUBLIC.NEWTRAININGDATA_CLEANED AS
SELECT
  -- normalize loan type
  REPLACE(
    REPLACE(
      REPLACE(LOAN_TYPE_NAME,
        'FSA/RHS-guaranteed', 'FSA_RHS'),
        'FHA-insured', 'FHA'),
        'VA-guaranteed', 'VA'
  ) AS LOAN_TYPE_NAME,

  -- normalize loan purpose
  REPLACE(
    REPLACE(
      REPLACE(LOAN_PURPOSE_NAME,
        '-', '_'),
        ' ', '_'),
        '.', '_'
  ) AS LOAN_PURPOSE_NAME,

  -- normalize county
  REPLACE(
    REPLACE(
      REPLACE(COUNTY_NAME,
        '-', ''),   -- remove hyphens
        '_', ''),   -- remove underscores
        '.', ''     -- remove periods
  ) AS COUNTY_NAME,

  WEEK_START_DATE,
  WEEK,
  LOAN_ID,
  TS,
  APPLICANT_INCOME_000S,
  LOAN_AMOUNT_000S,
  MORTGAGERESPONSE
FROM AICOLLEGE.PUBLIC.NEWTRAININGDATA;

### 🧪 Feature Engineering & Feature Store Setup

In this phase, we define and register a consistent set of engineered features using the cleaned `NEWTRAININGDATA_CLEANED` dataset as input. These features are transformed using **Snowflake ML preprocessing** and stored in the **Feature Store** for reuse across training and scoring workflows.

The **Snowflake Feature Store** enables:
- ✅ Centralized tracking of feature definitions, lineage, and versions  
- ♻️ Reuse of production-ready features across both training and inference pipelines  
- 🔍 Transparent, reproducible, and maintainable ML workflows

---

### What we’ll do:
1. Identify categorical features (`LOAN_TYPE_NAME`, `LOAN_PURPOSE_NAME`, `COUNTY_NAME`) and apply **One-Hot Encoding (OHE)** using `OneHotEncoder` from *snowflake.ml*.  
2. Keep the original raw columns for **transparency and lineage**.  
3. Fill in any missing values to ensure clean inputs.  
4. Write the transformed dataset to a persistent table `NEWTRAININGDATA_FINAL`.  
5. Optionally register engineered features in the Feature Store for consistent batch inference and monitoring.

> 🧱 Separating raw definitions from model-specific transformations gives us:
> - 🪄 Reusable and modular feature logic  
> - 🔁 Flexibility to experiment with alternative encoding strategies  
> - 🧪 Reliable inputs for training, production scoring, and explainability

> ⚠️ **NumPy 2.0 Note**:  
> Snowflake ML expects the `np.float_` alias, which was removed in NumPy ≥ 2.0. Add this shim before importing `OneHotEncoder`:
> ```python
> import numpy as np  
> if not hasattr(np, "float_"):  
>     np.float_ = np.float64  
> ```

In [ ]:
# --- Feature Engineering with OneHotEncoder in Snowflake ML ---
import numpy as np
from snowflake.snowpark.functions import col
from snowflake.snowpark.types import StringType, IntegerType, FloatType, DoubleType, DecimalType
from snowflake.ml.modeling.preprocessing import OneHotEncoder

# Temporary patch for NumPy >= 2.0
if not hasattr(np, 'float_'):
    np.float_ = np.float64

# Load cleaned training data
df_retrain = session.table("AICOLLEGE.PUBLIC.NEWTRAININGDATA_CLEANED")

# Identify categorical columns (exclude timestamp and label)
categorical_cols = [
    f.name
    for f in df_retrain.schema
    if isinstance(f.datatype, StringType)
       and f.name not in ("TS", "MORTGAGERESPONSE")
]

# Create output column names with _OHE suffix
encoded_cols = [f"{c}_OHE" for c in categorical_cols]

# Instantiate OneHotEncoder
ohe = OneHotEncoder(        # --> Use Snowflake's OneHotEncoder function
    input_cols=categorical_cols,
    output_cols=encoded_cols,
    drop_input_cols=False
)

# Apply encoder
df_encoded = ohe.fit(df_retrain).transform(df_retrain)

# Identify numeric columns only for safe fillna
numeric_cols = [
    f.name for f in df_encoded.schema
    if isinstance(f.datatype, (IntegerType, FloatType, DoubleType, DecimalType))
]

# Apply fillna only to numeric columns
df_encoded = df_encoded.fillna({col_name: 0 for col_name in numeric_cols})

# Save encoded data to Snowflake table
df_encoded.write.mode("overwrite").save_as_table("AICOLLEGE.PUBLIC.NEWTRAININGDATA_FINAL")  # --> Save as table AICOLLEGE.PUBLIC.NEWTRAININGDATA_FINAL

# Reload as a persisted DataFrame
df_encoded_persisted = session.table("AICOLLEGE.PUBLIC.NEWTRAININGDATA_FINAL")

### 🗄️ Register Loan Features with the Feature Store

Now that we’ve packaged our cleaned, one-hot encoded data (`NEWTRAININGDATA_FINAL`),  
we’ll register exactly the features our model needs in the **Snowflake Feature Store**.

Why this matters:
- 🎯 **Single source of truth** - the same features feed training, inference, and monitoring.
- 🔍 **Time‑aware joins & lineage** – every feature is versioned and timestamped.
- 🔄 **Model‑agnostic** – any future model can reuse the view without new ETL.

In this step, we will:
1. Define a `LOAN_ENTITY` on the `LOAN_ID` primary key  
2. Select only the OHE columns (those containing `"_OHE_"`) **plus** the numeric features  
   - `APPLICANT_INCOME_000S`  
   - `LOAN_AMOUNT_000S`  
   - `WEEK`  
3. **Exclude** any leakage columns (`MORTGAGERESPONSE`, `TS`, etc.)  
4. Register a Feature View (e.g. `LOAN_FEATURES` or `OHE_FEATURE_VIEW`)  
   using `WEEK_START_DATE` as the timestamp for time-travel joins

In [ ]:
# --- Register One-Hot Encoded Features with Snowflake Feature Store 
import warnings
from snowflake.ml.feature_store import FeatureStore, FeatureView, Entity, CreationMode
from snowflake.snowpark.functions import col

# 1) Read your encoded table
df_encoded = session.table("AICOLLEGE.PUBLIC.NEWTRAININGDATA_FINAL")

# 2) Identify raw feature columns
numeric_cols = ["APPLICANT_INCOME_000S", "LOAN_AMOUNT_000S", "WEEK"]
raw_ohe_cols = [c for c in df_encoded.columns if "_OHE_" in c]

# 3) Build a rename map to strip quotes & replace spaces/dots with underscores
def sanitize(name):
    return name.strip('"').replace(" ", "_").replace(".", "_")

rename_map = {c: sanitize(c) for c in ["LOAN_ID", "WEEK_START_DATE"] + raw_ohe_cols + numeric_cols}

# 4) Select & rename in one go
df_features = df_encoded.select([
    col(old).alias(new)
    for old, new in rename_map.items()
])

# 5) Initialize Feature Store
fs = FeatureStore(  # --> Use Snowflake's Feature Store (FeatureStore)
    session=session,
    database="AICOLLEGE",
    name="PUBLIC",
    default_warehouse="AICOLLEGE",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

# 6) Register entity
loan_entity = Entity(name="LOAN_ENTITY", join_keys=["LOAN_ID"])  # --> Save as LOAN_ENTITY as the Feature Store Entity
fs.register_entity(loan_entity)

# 7) Register Feature View
fv = FeatureView(  # --> Use Snowflake's Feature View (FeatureView)
    name="LOAN_FEATURES_OHE",  # --> Save as LOAN_FEATURES_OHE Feature View
    entities=[loan_entity],
    feature_df=df_features,
    timestamp_col="WEEK_START_DATE",  # --> Ensure you include your TIMESTAMP column with TIMESTAMP_NTZ format (WEEK_START_DATE)
    refresh_freq="1 day"  # --> Ensure your Snowflake Feature Store (dynamic table) has a valid refresh frequency value
)
fs.register_feature_view(fv, version="1", overwrite=True)
print("✅ Feature View registered with", len(rename_map), "features:", list(rename_map.values()))

In [ ]:
-- List dynamic tables created by the Feature Store for OHE_FEATURE_VIEW.
SHOW TABLES LIKE 'LOAN_FEATUREs_OHE%';

### 🔄 Retrain and Compare ML Models (Using OSS Libraries)

Now that our **one-hot encoded features** are saved in the `NEWTRAININGDATA_FINAL` table, we’ll retrain multiple candidate models using the enriched training dataset.

This step includes:

- **Loading** the feature-engineered data from Snowflake  
- Converting it to a **Pandas DataFrame** for modeling  
- Splitting into **train (80% ≈ 4,240 samples)** and **test (20% ≈ 1,060 samples)** sets  
- **Training** and **evaluating** three OSS-based models:  
  - XGBoost  
  - Random Forest  
  - Logistic Regression  
- Comparing their performance using **test set accuracy**

### ❗ Why use Pandas?

While your data lives in Snowflake, OSS libraries like `scikit-learn` and `xgboost` require **in-memory data**. They do **not support Snowpark DataFrames** directly.  
For that reason, we convert Snowflake data to a Pandas DataFrame before training models locally in the notebook.

This approach:
- ✅ Aligns with Snowflake’s guidance to use OSS libraries for modeling  
- ⚙️ Enables flexible development and model experimentation  
- 🔁 Keeps the pipeline compatible with Snowflake Model Registry and Batch Inference later

In [ ]:
# --- Model setup using only 'LOAN_PURPOSE_NAME_OHE' to reduce dimensionality ---
from snowflake.snowpark.functions import col
from snowflake.snowpark.types import FloatType
from sklearn.model_selection import train_test_split
import pandas as pd

# 1. Load OHE features — only keep what you need
df_features_all = session.table("AICOLLEGE.PUBLIC.LOAN_FEATURES_OHE$1")

# 2. Filter columns
ohe_cols_to_keep = [c for c in df_features_all.columns if c.startswith("LOAN_PURPOSE_NAME_OHE")]
selected_cols = ["LOAN_ID", "WEEK_START_DATE", "APPLICANT_INCOME_000S", "LOAN_AMOUNT_000S"] + ohe_cols_to_keep
df_features = df_features_all.select([col(c) for c in selected_cols])

# 3. Load labels
df_labels = session.table("AICOLLEGE.PUBLIC.NEWTRAININGDATA_FINAL") \
                   .select("LOAN_ID", "WEEK_START_DATE", "MORTGAGERESPONSE")

# 4. Join + cast numeric columns in Snowpark
df_joined = (
    df_features.join(df_labels, on=["LOAN_ID", "WEEK_START_DATE"])
    .with_column("APPLICANT_INCOME_000S", col("APPLICANT_INCOME_000S").cast(FloatType()))
    .with_column("LOAN_AMOUNT_000S", col("LOAN_AMOUNT_000S").cast(FloatType()))
    .to_pandas()
)

# 5. Clean column names and drop missing
df_joined.columns = df_joined.columns.str.strip('"').str.replace(" ", "_")
df_joined = df_joined.dropna()

# 6. Define X, y and keys
label = "MORTGAGERESPONSE"  # --> Which column is the target variable?
feature_cols = ohe_cols_to_keep + ["APPLICANT_INCOME_000S", "LOAN_AMOUNT_000S"]
X = df_joined[feature_cols]
y = df_joined[label].astype(int)
id_vec    = df_joined["LOAN_ID"].astype(int)
date_vec  = pd.to_datetime(df_joined["WEEK_START_DATE"])

# 7. Train/test split for all 4 arrays
X_train, X_test, y_train, y_test, id_train, id_test, date_train, date_test = train_test_split(
    X, y, id_vec, date_vec,
    test_size=0.2,
    random_state=42
)

# 8. Report
print(f"✅ Train set: {X_train.shape[0]} samples ({X_train.shape[0]/len(df_joined)*100:.1f}%)")
print(f"✅ Test  set: {X_test.shape[0]} samples ({X_test.shape[0]/len(df_joined)*100:.1f}%)")

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Scale your data
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)
X_test_scaled = pd.DataFrame(
    scaler.fit_transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

# Use a faster solver for large datasets
logreg_model = LogisticRegression(max_iter=1000, solver="saga")
rf_model     = RandomForestClassifier(n_estimators=50)  # reduce estimators for speed
xgb_model    = XGBClassifier(eval_metric="logloss", n_estimators=50)

# Train the models
logreg_model.fit(X_train_scaled, y_train)
rf_model.fit(X_train_scaled, y_train)
xgb_model.fit(X_train_scaled, y_train)

# Evaluate
print("🔍 Retrain Model Accuracy by candidate model:")
print(f"XGBoost:             {accuracy_score(y_test, xgb_model.predict(X_test_scaled)) * 100:.2f}%")
print(f"Random Forest:       {accuracy_score(y_test, rf_model.predict(X_test_scaled)) * 100:.2f}%")
print(f"Logistic Regression: {accuracy_score(y_test, logreg_model.predict(X_test_scaled)) * 100:.2f}%")

In [ ]:
from snowflake.snowpark.functions import col, row_number, sql_expr
from snowflake.snowpark.window import Window

# Build the prediction DataFrame as pandas
df_raw = pd.DataFrame({
    "LOAN_ID":            id_test.values,
    "WEEK_START_DATE":    date_test.values,
    "PREDICTED_RESPONSE": logreg_model.predict(X_test_scaled).astype(int),
    "PREDICTED_SCORE":    logreg_model.predict_proba(X_test_scaled)[:, 1],
    "MORTGAGERESPONSE":   y_test.values.astype(int),
})

# Convert to Snowpark DataFrame without passing schema
df_snowpark = session.create_dataframe(df_raw)

# Define a Snowpark window by WEEK_START_DATE with random sort
window_spec = Window.partition_by("WEEK_START_DATE").order_by(sql_expr('RANDOM()'))

# Assign row number for sampling
df_with_row_number = df_snowpark.with_column("row_num", row_number().over(window_spec))

# Filter to first 200 rows per WEEK_START_DATE
df_sampled = df_with_row_number.filter(col("row_num") <= 200).drop("row_num")

# Persist to Snowflake
df_sampled.write.mode("overwrite").save_as_table("AICOLLEGE.PUBLIC.V2_RAW_PREDICTIONS")

print("✅ V2 prediction sets persisted for Model Monitor.")

In [ ]:
CREATE OR REPLACE VIEW AICOLLEGE.PUBLIC.V2_PREDICTIONS AS
WITH b_deduped AS (
  SELECT *
  FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY LOAN_ID, WEEK_START_DATE ORDER BY TS DESC) AS rn
    FROM AICOLLEGE.PUBLIC.NEWTRAININGDATA_FINAL
  )
  WHERE rn = 1
)
SELECT
  b.WEEK_START_DATE,
  b.WEEK,
  b.LOAN_ID,
  b.TS,
  b.LOAN_TYPE_NAME,
  b.LOAN_PURPOSE_NAME,
  CAST(b.APPLICANT_INCOME_000S AS FLOAT) AS APPLICANT_INCOME_000S,
  b.LOAN_AMOUNT_000S,
  b.COUNTY_NAME,
  r.MORTGAGERESPONSE,
  r.PREDICTED_RESPONSE,
  r.PREDICTED_SCORE
FROM AICOLLEGE.PUBLIC.V2_RAW_PREDICTIONS AS r
JOIN b_deduped AS b
  USING (LOAN_ID, WEEK_START_DATE);

### 📦 Register Updated Model (Version 2)

We’ve retrained three OSS models (**XGBoost**, **Random Forest**, **Logistic Regression**) using the enriched `NEWTRAININGDATA` table.  
**Logistic Regression** achieved the best accuracy (~68%), so we will register this model in the **Snowflake Model Registry**.

> ✅ **Why register Logistic Regression?**
> 
> - Best performing model on updated data
> - Lightweight, interpretable, and easy to monitor
> - Aligns with OSS-first ML development best practices

We will register the model with:
- Full model artifact
- Version control for lifecycle management
- Metadata including training accuracy

> ⚠️ **Note:**  
> If you see a long OCSP-related warning like `fail-open to connect` or `Could not fetch OCSP response`, you can safely ignore it — the model will still register successfully. This warning is related to certificate checks and does **not** affect registration or functionality.


> ℹ️ **Note:**  
> We are **not yet assigning a production alias** (like `production` or `current_best`) — aliasing will happen after observability evaluation.

In [ ]:
# --- ✅ Register OSS Logistic Regression as V2 of Existing Model ---
from snowflake.ml.registry import Registry
from snowflake.ml.model import type_hints
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import logging

session  = get_active_session()
registry = Registry(session=session)

# Suppress verbose Snowflake connector logs
logging.getLogger("snowflake.connector").setLevel(logging.WARNING)
logging.getLogger("snowflake.ml").setLevel(logging.WARNING)
logging.basicConfig(level=logging.WARNING)

# Evaluate the Logistic Regression model
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
conf_matrix = confusion_matrix(y_test, y_pred).tolist()

# Sample input for registration
sample_input_data = X_train.copy().sample(n=5, random_state=42)

# Log as V2 of the original registered model
model_version = registry.log_model(
    model=xgb_model,
    model_name="COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL",  # >-- Provide required MLOPs HOL model name
    version_name="V2",                               # >-- Provide model version name (not V1)
    sample_input_data=sample_input_data,
    conda_dependencies=["scikit-learn"],
    comment="""Version 2: Logistic Regression model trained using OSS inside Snowflake.
This version supports Snowflake ML Observability and Explainability.""",
    metrics={
        "accuracy": accuracy,
        "f1_score": f1,
        "classification_report": report,
        "confusion_matrix": conf_matrix
    },
    task=type_hints.Task.TABULAR_BINARY_CLASSIFICATION,
    target_platforms=["WAREHOUSE"],  # >-- Ensures model is warehouse-compatible
    options={
        "enable_explainability": True,
        "method_options": {
            "predict": {"case_sensitive": True}
        }
    }
)

print("✅ Logistic Regression model registered as Version 2:", model_version.version_name)

In [ ]:
-- SEAI53: Validate model version V2 was registered in Snowflake Model Registry
SELECT util_db.public.se_grader(
  'SEAI53',
  (actual >= 1),
  actual,
  1,
  '✅ Model Version V2 successfully registered!'
) AS graded_results
FROM (
  SELECT COUNT(*) AS actual
  FROM AICOLLEGE.INFORMATION_SCHEMA.MODEL_VERSIONS
  WHERE MODEL_NAME = 'COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL'
    AND MODEL_VERSION_NAME = 'V2'
);

### 📈 Enable Model Monitoring with Snowflake ML Observability

Now that both **Version 1** and **Version 2** of our **XGBoost mortgage response model** are registered, we’ll use **Snowflake ML Observability** to track and compare their real-world performance over time.

---

### ✅ Why Model Monitoring Matters
- 📊 Track critical metrics like **accuracy**, **precision**, **recall**, and **F1 score**
- 🌀 Detect **prediction drift** and **feature drift** using recent inference batches
- 🧠 Establish a **baseline** for expected model behavior
- 📺 Monitor performance trends using **Snowsight’s built-in dashboards** — no external setup required

---

### 🔄 Why We’re Re-Creating Monitors for V1 and V2

We previously removed the original monitor for **Version 1**, but in order to use Snowsight’s **Compare Models** feature:

- Each model version must have its **own active monitor**
- We’ll now re-create both monitors to support side-by-side comparison between:
  - ✅ **Version 1** (original XGBoost model)
  - ✅ **Version 2** (retrained XGBoost with updated feature store data)

---

### 📂 Why Use `ALL_PREDICTIONS_WITH_GROUND_TRUTH`

To ensure monitoring reflects **real-world inference behavior**, we’ll base both monitors on the `ALL_PREDICTIONS_WITH_GROUND_TRUTH` table:
- ✅ Contains **actual predictions** made by deployed model versions
- ✅ Includes **ground truth labels** for evaluation
- ✅ Avoids leakage from training data or incomplete inference rows

---

This setup ensures transparent monitoring and lets us clearly assess whether Version 2 offers **consistent improvements** over Version 1 — not just in test performance, but in **ongoing model behavior in production**.


In [ ]:


ALTER TABLE AICOLLEGE.PUBLIC.ALL_PREDICTIONS_WITH_GROUND_TRUTH
RENAME COLUMN LOAN_AMOUNT_O80S TO LOAN_AMOUNT_000S;

In [ ]:
SELECT * FROM BASELINE_PREDICTIONS;

In [ ]:
-- Recreate ML Observability Model Monitor for XGBoost (V1) model.
CREATE OR REPLACE MODEL MONITOR MORTGAGE_MODEL_MONITOR_V1        -- Create for model version V1
WITH 
  MODEL = AICOLLEGE.PUBLIC.COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL,    -- Provide required MLOPs HOL model name
  VERSION = 'V1',
  FUNCTION = 'predict',
  SOURCE = AICOLLEGE.PUBLIC.ALL_PREDICTIONS_WITH_GROUND_TRUTH,   -- Full batch inference table
  BASELINE = AICOLLEGE.PUBLIC.BASELINE_PREDICTIONS,              -- Baseline model or training sample
  WAREHOUSE = AICOLLEGE,
  REFRESH_INTERVAL = '365 DAY',
  AGGREGATION_WINDOW = '7 DAYS',
  TIMESTAMP_COLUMN = WEEK_START_DATE,
  ID_COLUMNS = ('LOAN_ID'),
  PREDICTION_CLASS_COLUMNS = ('PREDICTED_RESPONSE'),
  ACTUAL_CLASS_COLUMNS = ('MORTGAGERESPONSE'),
  PREDICTION_SCORE_COLUMNS = ('PREDICTED_SCORE');

In [ ]:
-- Create new Model Monitor for the retrained XGBoost (V2) model.
CREATE OR REPLACE MODEL MONITOR MORTGAGE_MODEL_MONITOR_V2
WITH 
  MODEL = AICOLLEGE.PUBLIC.COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL,
  VERSION = 'V2',
  FUNCTION = 'predict',
  SOURCE = AICOLLEGE.PUBLIC.V2_PREDICTIONS,                -- <-- V2 training predictions
  BASELINE = AICOLLEGE.PUBLIC.BASELINE_PREDICTIONS,        -- <-- Baseline model
  WAREHOUSE = AICOLLEGE,
  REFRESH_INTERVAL = '365 DAY',
  AGGREGATION_WINDOW = '7 DAYS',
  TIMESTAMP_COLUMN = WEEK_START_DATE,
  ID_COLUMNS = ('LOAN_ID'),
  PREDICTION_CLASS_COLUMNS = ('PREDICTED_RESPONSE'),
  ACTUAL_CLASS_COLUMNS = ('MORTGAGERESPONSE'),
  PREDICTION_SCORE_COLUMNS = ('PREDICTED_SCORE');

### 🔍 Compare Model Versions in Snowsight

Now that monitors are set up for **Version 1** and **Version 2**, use Snowsight’s **Compare** feature to view them side-by-side.

1. In Snowsight, go to **AI & ML > Models**
2. Select **`COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL`**
3. Open either **V1** or **V2**
4. Under **Monitors**, select **`MORTGAGE_MODEL_MONITOR`**
5. Click the **“Compare”** toggle at the top of the monitor dashboard
6. Select both **V1** and **V2**
7. Set the date range to show **March 1, 2025 to July 20, 2025** of activity

Even though both versions use the same XGBoost model architecture, this comparison view helps validate consistency and monitor subtle trends across model versions.

### 🔍 Enable Model Explainability with Snowflake `EXPLAIN`

With our model now deployed and generating predictions, we can use **Snowflake’s built-in `EXPLAIN` function** to understand how input features contribute to model outputs — directly in Snowflake, without requiring any external libraries.

By calling the `EXPLAIN` function on a registered model version, we retrieve **feature contribution scores** (inspired by SHAP) that estimate how much each input influenced the model’s decision.

✅ Supported for models trained using **Snowflake ML** (e.g., XGBoost, LightGBM, Scikit-learn)  
✅ Provides per-feature contribution scores for individual predictions  
✅ Seamlessly integrated into **Python or SQL** workflows

In this section, we compute these contributions for a representative sample of our data and visualize the results for both **Version 1** and **Version 2** of our model — helping compare **which inputs drove predictions** across versions.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Step 1 — Run the EXPLAIN function on a sample input
model_version = registry.get_model("COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL").version("V2")
explanations_df = model_version.run(
    X_train.sample(n=50, random_state=42),
    function_name="EXPLAIN"  # --> Use Snowflake's new EXPLAIN function
)

# Step 2 — Identify explanation columns (excluding OHE columns)
explanation_cols = [
    col for col in explanations_df.columns
    if col.endswith("_explanation") and "_OHE_" not in col
]

# Step 3 — Horizontal Bar Plot (Snowflake Explain)
mean_abs_explanations = explanations_df[explanation_cols].abs().mean().sort_values(ascending=True)

plt.figure(figsize=(4, 2))
mean_abs_explanations.plot(kind="barh")
plt.title("SHAP-style Bar Plot (Snowflake EXPLAIN)")
plt.xlabel("Mean |Contribution|")
plt.ylabel("Feature")
plt.show()

# Step 4 — Beeswarm-style Dot Plot
dot_data = explanations_df[explanation_cols].copy()
dot_data.columns = [col.replace("_explanation", "") for col in dot_data.columns]
dot_data = dot_data.melt(var_name="Feature", value_name="Contribution")

plt.figure(figsize=(4, 2))
sns.stripplot(
    data=dot_data,
    x="Contribution",
    y="Feature",
    size=4,
    alpha=0.6,
    jitter=True,
    orient="h"
)
plt.title("Beeswarm-style Dot Plot (Snowflake EXPLAIN)")
plt.xlabel("Contribution Value")
plt.ylabel("Feature")
plt.show()

### 📊 Explainability Output (Snowflake `EXPLAIN` Function)

These charts show the **contribution of numeric input features** (excluding one-hot-encoded columns) as derived using Snowflake’s built-in `EXPLAIN` capability.

### 🔵 Top Chart: Mean Absolute Contribution (Bar Plot)
- Shows the **average absolute contribution** for each numeric feature across 50 examples.
- Longer bars indicate **greater overall influence** on the model’s prediction.
- In this case, `LOAN_AMOUNT_000S` has the largest impact, followed by `APPLICANT_INCOME_000S`.

### 🔴 Bottom Chart: Dot Plot (Beeswarm-style)
- Displays each example’s **raw contribution value** on the x-axis.
- The **vertical spread** shows variability across inputs; cluster density indicates concentration of effects.
- Useful to visualize how certain features push predictions **positively or negatively**.

Together, these plots improve transparency by surfacing **how much and in which direction** the input features are influencing predictions in your retrained model.

### ✅ Promote and Use Production Model with Aliases
Now that we’ve registered multiple versions of our model, we’ll adopt a lifecycle strategy using aliases — a flexible and production-ready approach supported by the **Snowflake Model Registry**.

Instead of hardcoding version numbers (e.g., V1, V2), we assign an alias like production to the latest validated model version. This enables us to:
- 🔄 Seamlessly promote new versions without updating downstream code
- 🔒 Maintain cleaner, version-agnostic pipeline logic
- ⚙️ Support rollback or staged rollout by shifting the alias

In [ ]:
-- Assign alias 'DECOMMISSIONED' to V1 for tracking purposes
ALTER MODEL COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL VERSION V1 SET ALIAS = DECOMMISSIONED;   -- Use Snowflake's ALIAS model parameter

-- Assign alias 'PRODUCTION' to V2
ALTER MODEL COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL VERSION V2 SET ALIAS = PRODUCTION;       -- Set V2 as alias = PRODUCTION

In [ ]:
-- SEAI54: Validate that Version V2 has the PRODUCTION alias
SELECT util_db.public.se_grader(
  'SEAI54',
  (actual >= 1),
  actual,
  1,
  '✅ Alias "PRODUCTION" correctly assigned to Version V2!'
) AS graded_results
FROM (
  SELECT COUNT(*) AS actual
  FROM AICOLLEGE.INFORMATION_SCHEMA.MODEL_VERSIONS
  WHERE MODEL_NAME = 'COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL'
    AND MODEL_VERSION_NAME = 'V2'
);

### 🔄 Roll Back Production Alias to V1

If you ever need to roll back your `production` alias from V2 back to V1, run these steps **after** your example pipelines:

```sql
-- 1) Remove the PRODUCTION alias from V2
ALTER MODEL AICOLLEGE.PUBLIC.COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL
  VERSION V2
  UNSET ALIAS;

-- 2) Clear any DECOMMISSIONED or other alias on V1
ALTER MODEL AICOLLEGE.PUBLIC.COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL
  VERSION V1
  UNSET ALIAS;

-- 3) Assign the PRODUCTION alias back to V1
ALTER MODEL AICOLLEGE.PUBLIC.COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL
  VERSION V1
  SET ALIAS = PRODUCTION;


### Score with the Promoted Production Model

With multiple model versions registered, we now use the **PRODUCTION** alias to serve both class labels and probability scores—without ever hard-coding a version number.

This example shows how to:

- Pull recent rows from the cleaned & encoded training data (`NEWTRAININGDATA_FINAL`)
- Join with the Feature Store (`LOAN_FEATURES_OHE$1`)
- Dynamically score new data using the latest production model, capturing:
  - 🔢 **SCORE**: the model’s probability that the positive class occurs  
  - 🔢 **PREDICTION**: the hard 0/1 label
- Persist a single table with both fields for downstream reporting or monitoring

By binding downstream code to the **PRODUCTION** alias you get:

- 🔄 **Version-agnostic pipelines** — no edits when you retrain or promote  
- 🔐 **Instant rollback** — simply retarget the alias if you discover an issue  
- ⚙️ **Plug-and-play automation** (Tasks, dbt, dynamic tables, etc.)

In [ ]:
from snowflake.ml.registry import Registry
from snowflake.snowpark.functions import col
from snowflake.snowpark import Session

# --- Initialize Registry and Get Model Version ---
reg = Registry(session=session, database_name="AICOLLEGE", schema_name="PUBLIC")
model_prod = reg.get_model("COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL").version("production")

# --- Load Feature Store Data ---
df_features = session.table("XXX")

# --- Drop rows with NULLs before inference ---
df_features_clean = df_features.dropna()

# --- Run prediction using PREDICT_PROBA ---
df_scores = model_prod.run(df_features_clean, function_name="PREDICT_PROBA")

# --- Safely check output feature names ---
print("Returned columns:", df_scores.columns)

# --- Build final prediction DataFrame ---
df_final = (
    df_scores
    .with_column("PROBABILITY_SCORE", col('"output_feature_1"'))  # use double quotes to handle Snowflake case sensitivity
    .with_column("PREDICTION_LABEL", (col('"output_feature_1"') > 0.5).cast("INT"))
)

# --- Save predictions ---
df_final.write.mode("overwrite").save_as_table("AICOLLEGE.PUBLIC.MORTGAGE_PREDICTIONS_PROD")

print("✅ Scoring pipeline completed successfully")

### 🧠 Feedback is Fuel  
We’d love your input to help improve this lab! Please take 3 minutes to fill out our feedback form:  
👉 [Submit Feedback Here](https://docs.google.com/forms/d/e/1FAIpQLSdJwMBSnYffjJI5N9db3IjHz36dMtcpvI1YmQuMwO0EgAUS8A/viewform?usp=header)  

### 🧹 HOL Cleanup

To reset your environment and avoid lingering monitors or scheduled alerts, you can drop the model monitor and alert below.

This is especially useful when:

- You're sharing an environment with other users
- You plan to rerun the notebook from the top
- You want to avoid refresh or alert-triggering compute

⚠️ Skip this section if you're actively monitoring your model in production.

In [ ]:
-- HOL CLEANUP SCRIPT

-- Drop model monitors first (they may reference the model)
DROP MODEL MONITOR IF EXISTS MORTGAGE_MODEL_MONITOR_V1;
DROP MODEL MONITOR IF EXISTS MORTGAGE_MODEL_MONITOR_V2;

-- -- Drop the ML model
DROP MODEL IF EXISTS COLLEGE_AI_HOL_XGB_MORTGAGE_MODEL;

-- Drop dynamic feature table (feature store)
DROP DYNAMIC TABLE IF EXISTS LOAN_FEATURES_OHE$1;

-- Drop the Feature View
DROP FEATURE VIEW AICOLLEGE.PUBLIC.LOAN_FEATURES_OHE VERSION 1;

-- Drop the Entity
DROP ENTITY AICOLLEGE.PUBLIC.LOAN_ENTITY;

-- Drop intermediate and prediction tables
DROP TABLE IF EXISTS ALL_PREDICTIONS_WITH_GROUND_TRUTH;
DROP TABLE IF EXISTS BASELINE_PREDICTIONS;
DROP TABLE IF EXISTS INFERENCEMORTGAGEDATA;
DROP TABLE IF EXISTS MORTGAGE_PREDICTIONS_PROD;
DROP TABLE IF EXISTS NEWTRAININGDATA;
DROP TABLE IF EXISTS NEWTRAININGDATA_CLEANED;
DROP TABLE IF EXISTS NEWTRAININGDATA_FINAL;
DROP TABLE IF EXISTS PREDICTIONS_WITH_GROUND_TRUTH;
DROP VIEW AICOLLEGE.PUBLIC.V2_PREDICTIONS;
DROP TABLE AICOLLEGE.PUBLIC.V2_RAW_PREDICTIONS;

-- Drop any custom file formats used (adjust name if needed)
DROP FILE FORMAT IF EXISTS MLOPS;